# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from A3C_Agent import *

2023-03-28 08:04:07.881963: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
ENV = "MountainCar-v0"
SUCESS_CRITERIA_VALUE = -100
SUCESS_CRITERIA_EPOCH = 100
EPISODES = 1000   

In [3]:
env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<MountainCarEnv<MountainCar-v0>>>>>

In [4]:
s = env.observation_space.sample()
s

array([-0.19969347,  0.02784247], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), (2,))

In [6]:
s = env.reset()[0]
env.action_space.n, env.action_space, env.step(1)

(3,
 Discrete(3),
 (array([-0.42616397, -0.00072561], dtype=float32), -1.0, False, False, {}))

# Advantage Asynch Actor-Critic 

This time we will implement the A3C not considering Actor and Critic as part of the same network

### Training/Hyperparam run

In [7]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [8]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_A3C/"

if TUNING_TYPE == "MANUAL":
    params = {}
    params["n_enviroment"] =[20]
    params["discount"] =[0.96, 0.97]
    params["end_of_episode"] = [400]
    params["learning_rate"] = [0.001]
    params["entropy_factor"] = [ 0.01, 0.05]
    params["epsilon"] = [1]
    params["boltzman_factor"] = [1]
    params["exploration_technique"] = ['soft', 'epsilon']
    params["training_steps"] = [3000000]
    params["dense_units"] = [32, 128]
    params["time_to_update"] = [400]
    params["use_LSTM"] =[False]


    hyperparam_combination = list(itertools.product(*list(params.values())))
    

    try:

        files = [name for name in os.listdir(logs_dir) if os.path.isfile(os.path.join(logs_dir, name)) and name != "logfile.txt" and name != "merged_results.json"]
        if len(files) >= 1: merge_JsonFiles(main_hyper_dir, logs_dir, files)

        res_file = logs_dir+"merged_results.json"
        def without_keys(d, keys):
            return {x: d[x] for x in d if x not in keys}


        if os.path.isfile(res_file):
            with open(res_file, 'r') as f:
                complete_file = json.load(f)

            newlist = sorted(complete_file, key=lambda d: d['mean_rewards'], reverse=True) 
            params = []
            for i, f in enumerate(newlist):
                label = "disc : "+str(f['discount'])+" | "+"lr : "+str(f['learning_rate'])+" | "+"entropy : "+str(f['entropy_factor'])+" | "+"update : "+str(f['time_to_update'])
                plt.figure(figsize=[35,4])
                plt.plot(f['rewards'], label = label)
                plt.legend()
                max_mean_reward = f['mean_rewards']
                params.append(f)
                print(without_keys(f,"rewards"))
                if i == 10:
                    break
            plt.title("Evaluation rewards"); plt.grid();
            plt.show()

    except:
        pass

    

In [9]:

if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_A3C"
    project_name = "keras_tunning_soft"

    tuner = kt.BayesianOptimization(
            MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_soft/"),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
    
    # project_name = "keras_tunning_epsilon"
    # tuner = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_epsilon/", exploration_tech = "epsilon"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 50,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name=project_name
    #     )
    # tuner.search(x=[0], y=[1])

    # project_name = "keras_tunning_boltzman"
    # tuner = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_boltzman/", exploration_tech = "boltzman"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 50,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name=project_name
    #     )
    # tuner.search(x=[0], y=[1])

    # dir = r"Hyperparam_kt_A3C_LSTM"
    # project_name = "keras_tunning_soft"
    # tuner_lstm = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name, writer = "logs_A3C_LSTM/A3C_soft/" ,use_LSTM = True),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 40,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name='keras_tunning'
    #     )
    # tuner_lstm.search(x=[0], y=[1])

    # project_name = "keras_tunning_epsilon"
    # tuner_lstm = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name, writer = "logs_A3C_LSTM/A3C_epsilon/" ,use_LSTM = True, exploration_tech = "epsilon"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 40,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name='keras_tunning'
    #     )
    # tuner_lstm.search(x=[0], y=[1])

    # project_name = "keras_tunning_boltzman"
    # tuner_lstm = kt.BayesianOptimization(
    #         MyHyperModel(hyper_dir = dir+"/"+project_name, writer = "logs_A3C_LSTM/A3C_boltzman/" ,use_LSTM = True, exploration_tech = "boltzman"),
    #         objective= kt.Objective('total_train_reward', direction="max"), 
    #         max_trials = 40,
    #         # distribution_strategy= strategy,
    #         directory=dir,
    #         project_name='keras_tunning'
    #     )
    # tuner_lstm.search(x=[0], y=[1])
   
else : 
    
    for exploration_tech in ['boltzman', 'epsilon']:
        print("Acquiring parameters ....")
        writer= "Training/fit_A3C/"

        n_enviroment = 10
        training_steps = 1000000
        learning_rate= 0.001
        entropy_factor = 0.1
        discount = 0.99
        dense_units = 512
        lstm_units = 128
        time_to_update= 100
        end_of_episode = 300
        use_LSTM = True

        
        run_training(training_steps, learning_rate, entropy_factor, exploration_tech, discount, time_to_update, dense_units, lstm_units, n_enviroment, writer, use_LSTM, end_of_episode)
            

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_soft/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [10]:
dir = r"Hyperparam_kt_A3C"
project_name = "keras_tunning_soft"
exploration_tech = "soft"
tuner = kt.BayesianOptimization(
        MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_soft/"),
        objective= kt.Objective('total_train_reward', direction="max"), 
        max_trials = 50,
        # distribution_strategy= strategy,
        directory=dir,
        project_name=project_name
    )

hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    n_env = 30
    end_ep = 1000
    ep = 1
    bolt_fact = 1
    train_steps = 5000000
    disc = trials.hyperparameters.values["discount"]
    lr = trials.hyperparameters.values["learning_rate"]
    entropy_fact = trials.hyperparameters.values["entropy_factor"]
    d = trials.hyperparameters.values["dense_units"]
    time_to_update = 100 if exploration_tech == "soft" else trials.hyperparameters.values["time_to_update"]
    ulstm = False

    hyperparam_combination.append((n_env, disc, end_ep, lr, entropy_fact, ep, bolt_fact, exploration_tech, train_steps, d, time_to_update, ulstm))

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_soft/tuner0.json
Trial id :32 | Score :-128.2 --> {'discount': 0.97, 'entropy_factor': 0.026470423848949327, 'learning_rate': 0.0013852930017904305, 'dense_units': 181}
Trial id :05 | Score :-134.54 --> {'discount': 0.97, 'entropy_factor': 0.025255076359307165, 'learning_rate': 0.0044706622554329545, 'dense_units': 113}
Trial id :02 | Score :-135.27 --> {'discount': 0.96, 'entropy_factor': 0.025779784100311638, 'learning_rate': 0.003991710454644242, 'dense_units': 119}


In [11]:
len(hyperparam_combination), hyperparam_combination

(3,
 [(30,
   0.97,
   1000,
   0.0013852930017904305,
   0.026470423848949327,
   1,
   1,
   'soft',
   5000000,
   181,
   100,
   False),
  (30,
   0.97,
   1000,
   0.0044706622554329545,
   0.025255076359307165,
   1,
   1,
   'soft',
   5000000,
   113,
   100,
   False),
  (30,
   0.96,
   1000,
   0.003991710454644242,
   0.025779784100311638,
   1,
   1,
   'soft',
   5000000,
   119,
   100,
   False)])

In [12]:
# run_hyperparam(TUNING_TYPE= "MANUAL", logs_dir=logs_dir,  main_hyper_dir = main_hyper_dir, conda_python_exec= conda_python_exec, py_file = "mountaincar_DDQN.py" ,TUNING_TYPE = TUNING_TYPE, hyperparam_combination = hyperparam_combination,  total_files = 4)

In [13]:

val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_A3C"

for exploration_tech in ["soft" ,"epsilon", "boltzman"]:

    project_name = "keras_tunning_"+exploration_tech

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_"+exploration_tech+"/", exploration_tech =exploration_tech),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            directory= dir,
            project_name=project_name
        )

    for trials in tuner.oracle.get_best_trials(num_trials=1):
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
        

    env_model = tuner.get_best_models()[0]
    final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./A3C_"+exploration_tech+"_video.mp4")
    print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

    

INFO:tensorflow:Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_soft/tuner0.json
Trial id :32 | Score :-128.2 --> {'discount': 0.97, 'entropy_factor': 0.026470423848949327, 'learning_rate': 0.0013852930017904305, 'dense_units': 181}
Trial number :  50
Moviepy - Building video ./A3C_soft_video.mp4.
Moviepy - Writing video ./A3C_soft_video.mp4



Moviepy - Done !
Moviepy - video ready ./A3C_soft_video.mp4


Final mean reward ' soft ': -129.105
INFO:tensorflow:Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_epsilon/tuner0.json
Trial id :26 | Score :-152.14 --> {'discount': 0.9700000000000001, 'entropy_factor': 0.04707560775392967, 'learning_rate': 0.0001844746208570615, 'dense_units': 101, 'time_to_update': 500}
Trial number :  50
Moviepy - Building video ./A3C_epsilon_video.mp4.
Moviepy - Writing video ./A3C_epsilon_video.mp4



Moviepy - Done !
Moviepy - video ready ./A3C_epsilon_video.mp4


Final mean reward ' epsilon ': -121.925
INFO:tensorflow:Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_boltzman/tuner0.json
Trial id :02 | Score :-130.9 --> {'discount': 0.96, 'entropy_factor': 0.012404128692054612, 'learning_rate': 0.0030824965995617138, 'dense_units': 169, 'time_to_update': 300}
Trial number :  50
Moviepy - Building video ./A3C_boltzman_video.mp4.
Moviepy - Writing video ./A3C_boltzman_video.mp4



Moviepy - Done !
Moviepy - video ready ./A3C_boltzman_video.mp4


Final mean reward ' boltzman ': -138.91
